In [1]:
threshold = 0.005  # i.e., 0.5% movement


In [4]:
import pandas as pd

# Load your dataset with FinBERT sentiment
df = pd.read_csv("../data/features_btc_with_sentiment.csv", parse_dates=["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# Step 1: Calculate price change percentage
df['close_pct_change'] = df['Close'].pct_change()

# Step 2: Define thresholds
threshold = 0.005  # 0.5% up/down

def synthetic_sentiment(change):
    if pd.isna(change):
        return 0
    elif change > threshold:
        return 1
    elif change < -threshold:
        return -1
    else:
        return 0

# Step 3: Create a new column for fallback sentiment
df['synthetic_fallback'] = df['close_pct_change'].apply(synthetic_sentiment)

# Step 4: Inject synthetic only where sentiment is missing or zero
df['daily_sentiment'] = df.apply(
    lambda row: row['synthetic_fallback'] if (pd.isna(row['daily_sentiment']) or row['daily_sentiment'] == 0) else row['daily_sentiment'],
    axis=1
)

# Cleanup
df.drop(columns=['close_pct_change', 'synthetic_fallback'], inplace=True)

# Save final dataset
df.to_csv("../data/features_btc_with_hybrid_sentiment.csv", index=False)
